<a href="https://colab.research.google.com/github/SATYAJEET323/Property-Valuation-model/blob/main/Tech_NLP_Mini_Project_Part_3_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. **Data Loading**:

Load the dataset (CSV format in this case).

Perform exploratory data analysis (EDA) to understand class distributions and data structure.

Split the dataset into training and validation sets.

In [ ]:
# Install simpletransformers package
!pip install simpletransformers

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
import pandas as pd

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path to your dataset in Google Drive
file_path = '/content/drive/My Drive/NLP_dataset/Property_val_dataset.csv'  # Change this to the actual path

# Step 3: Load the dataset (assuming it's a CSV file)
df = pd.read_csv(file_path)

# Step 4: Display the dataset to check
print(df.head())  # Show the first few rows of the dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                               Input Prediction
0  Location: Delhi, Dwarka; Size: 3030 sq. ft; Am...       High
1  Location: Bangalore, Electronic City; Size: 82...        Low
2  Location: Pune, Hinjewadi; Size: 2798 sq. ft; ...       High
3  Location: Kolkata, Park Street; Size: 4634 sq....       High
4  Location: Delhi, Dwarka; Size: 3844 sq. ft; Am...       High


In [ ]:
# Exploratory Data Analysis (EDA)
print(df.info())  # Overview of data structure
print(df['Prediction'].value_counts())  # Class distribution

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Input       1000 non-null   object
 1   Prediction  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB
None
Prediction
Medium    486
High      311
Low       203
Name: count, dtype: int64


In [ ]:
# Split dataset into train and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Preparing the data in the correct format for SimpleTransformers
train_df = pd.DataFrame({
    'Prediction': train_data['Prediction'],
    'Prediction': train_data['Prediction']
})

val_df = pd.DataFrame({
    'Prediction': val_data['Prediction'],
    'Prediction': val_data['Prediction']
})


In [ ]:
train_df['Prediction'] = train_df['Prediction'].astype(str)
val_df['Prediction'] = val_df['Prediction'].astype(str)

In [ ]:
# Define the label mapping dictionary before using it
label_mapping = {
    'positive': 1,
    'negative': 0,
    'neutral': 2
}

# Convert text labels to numeric values
train_df = pd.DataFrame({
    'text': train_data['Input'],
    'labels': train_data['Prediction'].map(label_mapping)
})

val_df = pd.DataFrame({
    'text': val_data['Input'],
    'labels': val_data['Prediction'].map(label_mapping)
})


2. **Text Processing**:

Here we clean the text by removing special characters, converting to lowercase, removing numbers, and stripping any extra whitespace.

In [ ]:
import re

# Define a function to clean text data
def clean_text(text):
    if isinstance(text, str):  # Ensure text is a string
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = text.strip()
    return text

# Convert 'labels' column to string type and apply cleaning
train_df['labels'] = train_df['labels'].astype(str).apply(clean_text)
val_df['labels'] = val_df['labels'].astype(str).apply(clean_text)

print(train_df.head())  # Check output


                                                  text labels
29   Location: Kolkata, New Town; Size: 3387 sq. ft...    nan
535  Location: Pune, Koregaon Park; Size: 4448 sq. ...    nan
695  Location: Ahmedabad, Satellite; Size: 894 sq. ...    nan
557  Location: Hyderabad, Banjara Hills; Size: 3993...    nan
836  Location: Mumbai, South Mumbai; Size: 1827 sq....    nan


3. **Text Embedding using BERT and RoBERTa:**

Use BERT and RoBERTa models for embedding the cleaned text. These models automatically tokenize and embed the text.

In [ ]:
from simpletransformers.classification import ClassificationModel

# Create a BERT model for text classification
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, use_cuda=False)  # Set use_cuda=True if using a GPU

# Create a RoBERTa model for text classification
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda=False)  # Set use_cuda=True if using a GPU

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight

4. **Model Training with BERT and RoBERTa**:

In [ ]:
# Preparing the data in the correct format for SimpleTransformers
# Assuming 'Prediction' column contains the text and you want to predict the same 'Prediction'
# If you have a separate column for labels, replace 'Prediction' with that column name

# Create a dictionary to map unique string labels to numerical labels starting from 0
unique_labels = train_data['Prediction'].unique()
num_labels = len(unique_labels)  # Get the actual number of unique labels

label_mapping = {label: idx for idx, label in enumerate(unique_labels)}


train_df = pd.DataFrame({
    'text': train_data['Input'],  # Changed 'Prediction' to 'text' for the input text
    'labels': train_data['Prediction'].map(label_mapping)  # Map string labels to numerical labels
})

val_df = pd.DataFrame({
    'text': val_data['Input'],  # Changed 'Prediction' to 'text' for the input text
    'labels': val_data['Prediction'].map(label_mapping)  # Map string labels to numerical labels
})

# Update the model definition with the correct number of labels
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=num_labels, use_cuda=False)  # Update num_labels

# Now you can train your model
bert_model.train_model(train_df)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/100 [00:00<?, ?it/s]

(100, 1.0529081898927688)

In [ ]:
!rm -rf outputs/

In [ ]:
# Import necessary libraries
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path to your dataset in Google Drive
file_path = '/content/drive/My Drive/NLP_dataset/Property_val_dataset.csv'  # Change this to the actual path

# Step 3: Load the dataset (assuming it's a CSV file)
df = pd.read_csv(file_path)
# Exploratory Data Analysis (EDA)
print(df.info())
print(df['Prediction'].value_counts())

# Split dataset into train and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Create label mapping from the full dataset
unique_labels = df['Prediction'].unique()
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

# Convert text and labels into DataFrame
train_df = pd.DataFrame({
    'text': train_data['Input'],
    'labels': train_data['Prediction'].map(label_mapping)
})

val_df = pd.DataFrame({
    'text': val_data['Input'],
    'labels': val_data['Prediction'].map(label_mapping)
})

# Ensure 'Prediction/' is clean before training
output_dir = "outputs/"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

# Define the model correctly
roberta_model = ClassificationModel(
    model_type='roberta',
    model_name='roberta-base',
    num_labels=len(label_mapping),
    use_cuda=False
)

# Train the model
roberta_model.train_model(train_df, overwrite_output_dir=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Input       1000 non-null   object
 1   Prediction  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB
None
Prediction
Medium    486
High      311
Low       203
Name: count, dtype: int64


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/100 [00:00<?, ?it/s]

(100, 1.04779947578907)

In [ ]:
!rm -rf outputs/

In [ ]:
# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path to your dataset in Google Drive
file_path = '/content/drive/My Drive/NLP_dataset/Property_val_dataset.csv'  # Change this to the actual path

# Step 3: Load the dataset (assuming it's a CSV file)
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install simpletransformers # Install the simpletransformers library

5. **Evaluation on Validation Set:**

Evaluate the performance of both BERT and RoBERTa models on the validation set using accuracy, precision, recall, and F1-score.

In [ ]:
from simpletransformers.classification import ClassificationArgs, ClassificationModel
import time
import shutil
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset (replace with your dataset path)

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path to your dataset in Google Drive
file_path = '/content/drive/My Drive/NLP_dataset/Property_val_dataset.csv'  # Change this to the actual path

# Step 3: Load the dataset (assuming it's a CSV file)
df = pd.read_csv(file_path)

# Exploratory Data Analysis (EDA)
print(df.info())  # Overview of data structure
print(df['Prediction'].value_counts())  # Class distribution

# Split dataset into train and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Preparing the data in the correct format for SimpleTransformers
# Assuming 'Output' column contains the text and you want to predict the same 'Output'
# If you have a separate column for labels, replace 'Output' with that column name

# Create a dictionary to map unique string labels to numerical labels starting from 0
unique_labels = train_data['Prediction'].unique()
num_labels = len(unique_labels)  # Get the actual number of unique labels

label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

train_df = pd.DataFrame({
    'text': train_data['Input'],  # Changed 'Output' to 'text' for the input text
    'labels': train_data['Prediction'].map(label_mapping)  # Map string labels to numerical labels
})

val_df = pd.DataFrame({
    'text': val_data['Input'],  # Changed 'Output' to 'text' for the input text
    'labels': val_data['Prediction'].map(label_mapping)  # Map string labels to numerical labels
})


# Set up model arguments with custom hyperparameters
model_args = ClassificationArgs(
    num_train_epochs=3,       # Start with 3 epochs
    train_batch_size=8,       # Use a batch size of 8
    eval_batch_size=8,        # Same for evaluation
    learning_rate=3e-5,       # Learning rate
    max_seq_length=128,       # Max sequence length
    weight_decay=0.01,        # Weight decay
    warmup_steps=0,           # Optional: adjust based on total steps
    logging_steps=50,         # Log training progress every 50 steps
    save_steps=200,           # Save the model every 200 steps
)

# Function to clear the outputs directory
def clear_outputs_directory():
  # Try to remove the directory using shutil.rmtree
  try:
    shutil.rmtree("outputs/")
  except OSError as e:
    print(f"Error removing outputs/ directory: {e}")
  # Wait for 2 seconds to allow any processes to release the directory
  time.sleep(2)
  # Create the directory if it doesn't exist
  os.makedirs("outputs/", exist_ok=True)


# Train the BERT model
clear_outputs_directory()
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=num_labels, args=model_args, use_cuda=False)
bert_model.train_model(train_df, overwrite_output_dir=True)

# Train the RoBERTa model
clear_outputs_directory()
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=num_labels, args=model_args, use_cuda=False)
roberta_model.train_model(train_df, overwrite_output_dir=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Input       1000 non-null   object
 1   Prediction  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB
None
Prediction
Medium    486
High      311
Low       203
Name: count, dtype: int64
Error removing outputs/ directory: [Errno 2] No such file or directory: 'outputs/'


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/100 [00:00<?, ?it/s]

(300, 1.0412959591547648)

In [ ]:
# Evaluate BERT on validation data
result_bert, model_outputs_bert, wrong_predictions_bert = bert_model.eval_model(val_df)

print("BERT Evaluation Results:")
print(result_bert)

0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

BERT Evaluation Results:
{'mcc': 0.0, 'eval_loss': 1.0639321208000183}


In [ ]:
# Evaluate RoBERTa on validation data
result_roberta, model_outputs_roberta, wrong_predictions_roberta = roberta_model.eval_model(val_df)

print("RoBERTa Evaluation Results:")
print(result_roberta)

0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

RoBERTa Evaluation Results:
{'mcc': 0.0, 'eval_loss': 1.015165536403656}


**6. Saving the Best Model**

In [ ]:
bert_model.save_model('bert_best_model')

In [ ]:
roberta_model.save_model('roberta_best_model')

In [ ]:
!ls -l ./bert_best_model


total 428652
drwxr-xr-x 2 root root      4096 Mar 30 14:30 checkpoint-1-epoch-1
drwxr-xr-x 2 root root      4096 Mar 30 14:30 checkpoint-2-epoch-2
drwxr-xr-x 2 root root      4096 Mar 30 14:30 checkpoint-3-epoch-3
-rw-r--r-- 1 root root       687 Mar 30 14:37 config.json
-rw-r--r-- 1 root root      2660 Mar 30 14:37 model_args.json
-rw-r--r-- 1 root root 437958648 Mar 30 14:37 model.safetensors
-rw-r--r-- 1 root root       125 Mar 30 14:37 special_tokens_map.json
-rw-r--r-- 1 root root      1221 Mar 30 14:37 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Mar 30 14:37 tokenizer.json
-rw-r--r-- 1 root root      3704 Mar 30 14:37 training_args.bin
-rw-r--r-- 1 root root    231508 Mar 30 14:37 vocab.txt


**7. Prediction on Real-World Input**:

In [ ]:
!pip install simpletransformers


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

# Define the number of labels (ensure this is correctly set according to your dataset)
num_labels = 2  # Change this based on your classification task

# Set up model arguments
model_args = ClassificationArgs(
    num_train_epochs=3,
    train_batch_size=8,
    eval_batch_size=8,
    learning_rate=3e-5,
    max_seq_length=128,
    weight_decay=0.01,
    warmup_steps=0,
    logging_steps=50,
    save_steps=200,
    overwrite_output_dir=True,  # Ensure previous files are replaced
    output_dir='./bert_best_model'  # Specify save path explicitly
)

# Example training data (replace with actual training data)
train_data = {
    'text': ['example sentence 1', 'example sentence 2'],
    'labels': [0, 1]  # Ensure labels are numerical
}
train_df = pd.DataFrame(train_data)

# Create and train the BERT model
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=num_labels, args=model_args, use_cuda=False)
bert_model.train_model(train_df)

# Now load the model using the same path
bert_model = ClassificationModel('bert', './bert_best_model', use_cuda=False)

# Continue with your predictions
real_world_text = ['Bangalore', '3935 sq. ft', 'Rooftop access', 'Old house', 'gym']
predictions_bert, _ = bert_model.predict(real_world_text)
print(f"BERT Predictions: {predictions_bert}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Running Epoch 2 of 3:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

BERT Predictions: [0, 0, 0, 0, 0]


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
from sklearn.model_selection import train_test_split

# Example training data (replace with actual training data)
train_data = {
    'text': ['example sentence 1', 'example sentence 2'],
    'labels': [0, 1]  # Ensure labels are numerical and match correct column name
}
train_df = pd.DataFrame(train_data)

# Ensure num_labels is calculated correctly
unique_labels = train_df['labels'].unique()  # Updated from 'Output' to 'labels'
num_labels = len(unique_labels)  # Get the actual number of unique labels

# Set up model arguments
model_args = ClassificationArgs(
    num_train_epochs=3,
    train_batch_size=8,
    eval_batch_size=8,
    learning_rate=3e-5,
    max_seq_length=128,
    weight_decay=0.01,
    warmup_steps=0,
    logging_steps=50,
    save_steps=200,
    overwrite_output_dir=True,
    output_dir='./bert_best_model'
)

# Create and train the BERT model
bert_model = ClassificationModel(
    'bert',
    'bert-base-uncased',
    num_labels=num_labels,
    args=model_args,
    use_cuda=False
)  # ✅ Fixed missing parenthesis

bert_model.train_model(train_df)

# Now load the model using the same path
bert_model = ClassificationModel('bert', './bert_best_model', use_cuda=False)

# Continue with your predictions
real_world_text = ['Bangalore', '3935 sq. ft', 'Rooftop access', 'Old house', 'gym']
predictions_bert, _ = bert_model.predict(real_world_text)
print(f"BERT Predictions: {predictions_bert}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0it [00:00, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

BERT Predictions: [1, 0, 0, 0, 0]


In [3]:
!pip install simpletransformers

In [ ]:
!ls -l ./roberta_best_model


In [5]:
# Install simpletransformers in the same cell
!pip install simpletransformers
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
import shutil
import os
# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path to your dataset in Google Drive
file_path = '/content/drive/My Drive/NLP_dataset/Property_val_dataset.csv'  # Change this to the actual path

# Step 3: Load the dataset (assuming it's a CSV file)
df = pd.read_csv(file_path)

# Exploratory Data Analysis (EDA)
print(df.info())
print(df['Prediction'].value_counts())

# Split dataset into train and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Prepare data for SimpleTransformers
unique_labels = train_data['Prediction'].unique()
num_labels = len(unique_labels)

label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

train_df = pd.DataFrame({
    'text': train_data['Input'],
    'labels': train_data['Prediction'].map(label_mapping)
})

val_df = pd.DataFrame({
    'text': val_data['Input'],
    'labels': val_data['Prediction'].map(label_mapping)
})

# ----------------------
# RoBERTa Model Training and Saving
# ----------------------

# Remove the output directory if it exists
output_dir = './roberta_best_model'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

# Create and train the RoBERTa model
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=num_labels, use_cuda=False)
roberta_model.train_model(train_df, output_dir=output_dir) #Removed overwrite_output_dir=True,



# ----------------------
# RoBERTa Model Loading and Prediction
# ----------------------

# Load the saved RoBERTa model using the correct path
roberta_model = ClassificationModel('roberta', output_dir, use_cuda=False)

# Real-world input text
real_world_text = ["This is a great product!", "I didn't like the service."]

# Predict the class
predictions_roberta, _ = roberta_model.predict(real_world_text)

print(f"RoBERTa Predictions: {predictions_roberta}")

NameError: name 'drive' is not defined

In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00
   ━

In [1]:
# prediction on real-world input
import torch
from simpletransformers.classification import ClassificationModel
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder

def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r'[^a-zA-Z\s]', '', txt)
    txt = txt.strip()
    return txt

# load saved models; use use_cuda false if no gpu
# Changed 'bert_model' to './bert_best_model' to load the saved model from the correct directory
bert_model = ClassificationModel("bert", "./bert_best_model", use_cuda=False)
# Changed 'roberta_model' to './roberta_best_model' to load the saved model from the correct directory
roberta_model = ClassificationModel("roberta", "./roberta_best_model", use_cuda=False)

# set label encoder classes manually
encoder = LabelEncoder()
encoder.classes_ = np.array(['Bangalore','3935 sq. ft','Rooftop access','Old house','gym'])

input_text = input("enter product description: ")
cleaned_text = clean_text(input_text)

bert_pred, _ = bert_model.predict([cleaned_text])
roberta_pred, _ = roberta_model.predict([cleaned_text])

bert_lbl = encoder.inverse_transform(np.array(bert_pred))[0]
roberta_lbl = encoder.inverse_transform(np.array(roberta_pred))[0]

print("\nbert predicted category:", bert_lbl)
print("roberta predicted category:", roberta_lbl)

ModuleNotFoundError: No module named 'simpletransformers'

In [ ]:
# prediction on real-world input
import torch
from simpletransformers.classification import ClassificationModel
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder

def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r'[^a-zA-Z\s]', '', txt)
    txt = txt.strip()
    return txt



# set label encoder classes manually
encoder = LabelEncoder()
encoder.classes_ = np.array(['Bangalore','3935 sq. ft','Rooftop access','Old house','gym'])

input_text = input("enter product description: ")
cleaned_text = clean_text(input_text)

bert_pred, _ = bert_model.predict([cleaned_text])
roberta_pred, _ = roberta_model.predict([cleaned_text])

bert_lbl = encoder.inverse_transform(np.array(bert_pred))[0]
roberta_lbl = encoder.inverse_transform(np.array(roberta_pred))[0]

print("\nbert predicted category:", bert_lbl)
print("roberta predicted category:", roberta_lbl)

---

### **Text Classification by Fine-tuning Language Model**

---

### 1. **Data Loading**
   - Load the dataset (CSV format in this case).
   - Perform exploratory data analysis (EDA) to understand class distributions and data structure.
   - Split the dataset into training and validation sets.

```python
# Install simpletransformers package
!pip install simpletransformers

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset (replace with your dataset path)
data = pd.read_csv('text_classification_data.csv')

# Exploratory Data Analysis (EDA)
print(data.info())  # Overview of data structure
print(data['label'].value_counts())  # Class distribution

# Split dataset into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Preparing the data in the correct format for SimpleTransformers
train_df = pd.DataFrame({
    'text': train_data['text'],
    'labels': train_data['label']
})

val_df = pd.DataFrame({
    'text': val_data['text'],
    'labels': val_data['label']
})
```

---

### 2. **Text Processing**
   - Here we clean the text by removing special characters, converting to lowercase, removing numbers, and stripping any extra whitespace.

```python
import re

# Define a function to clean text data
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = text.strip()
    
    return text

# Apply the cleaning function to the dataset
train_df['text'] = train_df['text'].apply(clean_text)
val_df['text'] = val_df['text'].apply(clean_text)

print(train_df.head())
```

---

### 3. **Text Embedding using BERT and RoBERTa**
   - Use BERT and RoBERTa models for embedding the cleaned text. These models automatically tokenize and embed the text.

```python
from simpletransformers.classification import ClassificationModel

# Create a BERT model for text classification
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, use_cuda=False)  # Set use_cuda=True if using a GPU

# Create a RoBERTa model for text classification
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda=False)  # Set use_cuda=True if using a GPU
```

---

### 4. **Model Training with BERT and RoBERTa**

#### **Basic Model Training**

#### **Train the BERT Model**
```python
# Train BERT model
bert_model.train_model(train_df)
```

#### **Train the RoBERTa Model**
```python
# Train RoBERTa model
roberta_model.train_model(train_df)
```

#### **Model Training with Hyperparameters**
   - Train the models with a set of hyperparameters such as learning rate, batch size, epochs, etc.

```python
from simpletransformers.classification import ClassificationArgs

# Set up model arguments with custom hyperparameters
model_args = ClassificationArgs(
    num_train_epochs=3,       # Start with 3 epochs
    train_batch_size=8,       # Use a batch size of 8
    eval_batch_size=8,        # Same for evaluation
    learning_rate=3e-5,       # Learning rate
    max_seq_length=128,       # Max sequence length
    weight_decay=0.01,        # Weight decay
    warmup_steps=0,           # Optional: adjust based on total steps
    logging_steps=50,         # Log training progress every 50 steps
    save_steps=200,           # Save the model every 200 steps
)

# Train the BERT model with custom hyperparameters
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, args=model_args, use_cuda=False)
bert_model.train_model(train_df)

# Train the RoBERTa model with custom hyperparameters
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=model_args, use_cuda=False)
roberta_model.train_model(train_df)
```

---

### 5. **Evaluation on Validation Set**
   - Evaluate the performance of both BERT and RoBERTa models on the validation set using accuracy, precision, recall, and F1-score.

#### **Evaluate BERT Model**
```python
# Evaluate BERT on validation data
result_bert, model_outputs_bert, wrong_predictions_bert = bert_model.eval_model(val_df)

print("BERT Evaluation Results:")
print(result_bert)
```

#### **Evaluate RoBERTa Model**
```python
# Evaluate RoBERTa on validation data
result_roberta, model_outputs_roberta, wrong_predictions_roberta = roberta_model.eval_model(val_df)

print("RoBERTa Evaluation Results:")
print(result_roberta)
```

---

### 6. **Saving the Best Model**
   - Save the best-performing model for later use.

#### **Saving the BERT Model**
```python
bert_model.save_model('bert_best_model')
```

#### **Saving the RoBERTa Model**
```python
roberta_model.save_model('roberta_best_model')
```

---

### 7. **Prediction on Real-World Input**
   - Test the saved model on real-world input data. Preprocess the input text, use the model to predict the class, and output the results.

#### **Prediction Using BERT Model**
```python
# Load the saved BERT model
bert_model = ClassificationModel('bert', 'bert_best_model', use_cuda=False)

# Real-world input text
real_world_text = ["This is a great product!", "I didn't like the service."]

# Predict the class
predictions_bert, _ = bert_model.predict(real_world_text)

print(f"BERT Predictions: {predictions_bert}")
```

#### **Prediction Using RoBERTa Model**
```python
# Load the saved RoBERTa model
roberta_model = ClassificationModel('roberta', 'roberta_best_model', use_cuda=False)

# Real-world input text
real_world_text = ["This is a great product!", "I didn't like the service."]

# Predict the class
predictions_roberta, _ = roberta_model.predict(real_world_text)

print(f"RoBERTa Predictions: {predictions_roberta}")
```

---